In [ ]:
import pandas as pd
import os
import numpy as np

In [ ]:

def simple_combine_sheets(excel_file_path):
    """combine all sheets into one"""

    # Read all sheets
    all_sheets = pd.read_excel(excel_file_path, sheet_name=None)

    combined_data = []

    for sheet_name, df in all_sheets.items():
        # remove empty rows
        df_clean = df.dropna(how='all')

        # Add sheet name as year if not exists
        if 'Year' not in df_clean.columns:
            df_clean['Year'] = sheet_name

        combined_data.append(df_clean)

    # Combine all
    final_df = pd.concat(combined_data, ignore_index=True)

    # Save combined data
    output_file = "combined_world_cup_standings.xlsx"
    final_df.to_excel(output_file, index=False)

    print(f"Combined {len(all_sheets)} sheets into {output_file}")
    print(f"Final size: {final_df.shape}")

    return final_df

if __name__ == "__main__":
    file = "world_cup_standings.xlsx"

    if os.path.exists(file):
        df = simple_combine_sheets(file)
        print("\nFirst few rows:")
        print(df.head())
    else:
        print(f"File {file} not found!")

In [ ]:

def aggregate_team_stats(combined_file_path, output_file='aggregated_team_stats.xlsx'):

    df = pd.read_excel(combined_file_path)

    '''  # Display original team counts
    team_col = identify_team_column(df)
    print(f"\n Original team appearances:")
    team_counts = df[team_col].value_counts()
    for team, count in team_counts.head(10).items():
        print(f"{team}: {count} World Cups")'''

    # Identify numeric columns (these will be summed)
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()

    # Remove non-summable numeric columns
    non_summable = ['Year', 'Rk', 'Rk.', 'Rank', '#', 'No', 'Jersey Number']
    numeric_cols = [col for col in numeric_cols if col not in non_summable]


    # Identify text columns (these will be handled differently)
    text_cols = df.select_dtypes(include=['object']).columns.tolist()

    # Create aggregation rules
    aggregation_rules = {}

    # Sum numeric columns (stats)
    for col in numeric_cols:
        aggregation_rules[col] = 'sum'

    # Handle text columns
    for col in text_cols:
        if col == team_col:
            aggregation_rules[col] = 'first'  # Keep team name
        else:
            # For other text columns, show list of values or count
            aggregation_rules[col] = lambda x: ', '.join(map(str, x.unique())) if x.nunique() > 1 else x.iloc[0]


    aggregated_df = df.groupby(team_col, as_index=False).agg(aggregation_rules)

    # Calculate averages for key metrics
    aggregated_df = calculate_averages(aggregated_df, numeric_cols)

    print(f"Aggregation complete!")
    print(f"Original: {len(df)} rows")
    print(f"Aggregated: {len(aggregated_df)} unique teams")

    # Save results
    aggregated_df.to_excel(output_file, index=False)
    print(f"Saved aggregated data to: {output_file}")

    return aggregated_df

def identify_team_column(df):
    """Identify which column contains team names"""
    team_columns = ['squad', 'team', 'country', 'nation', 'player', 'name']

    for col in df.columns:
        col_lower = str(col).lower()
        for team_col in team_columns:
            if team_col in col_lower:
                return col

    # Fallback: first string column
    for col in df.columns:
        if df[col].dtype == 'object':
            return col

    return df.columns[0]

def calculate_averages(df, numeric_cols):
    """Calculate per-game averages for key statistics"""

    # Common columns that might represent games played
    games_columns = ['mp', 'matches', 'games', 'played', 'gp', 'm']

    games_col = None
    for col in df.columns:
        if str(col).lower() in games_columns and col in numeric_cols:
            games_col = col
            break

    if games_col and games_col in df.columns:
        print(f"Using '{games_col}' for per-game averages")

        # Columns to calculate averages for (avoid ratios and percentages)
        avg_columns = []
        exclude_keywords = ['%', 'percent', 'per', 'avg', 'average', 'rate', 'ratio']

        for col in numeric_cols:
            col_lower = str(col).lower()
            if (col != games_col and
                not any(keyword in col_lower for keyword in exclude_keywords) and
                df[col].sum() > 0):  # Only for columns with data
                avg_columns.append(col)

        # Calculate per-game averages
        for col in avg_columns[:10]:  # Limit to first 10 to avoid too many columns
            avg_col_name = f'{col}_per_game'
            df[avg_col_name] = df[col] / df[games_col]
            df[avg_col_name] = df[avg_col_name].round(2)

        print(f" Added per-game averages for {len(avg_columns[:10])} statistics")

    return df

def analyze_aggregated_data(agg_df):
    """Provide analysis of the aggregated data"""
    team_col = identify_team_column(agg_df)

    print(f"\nAGGREGATED DATA ANALYSIS")
    print("=" * 50)

    # Top teams by appearances
    if 'Appearances' in agg_df.columns:
        print(f"\nTeams with most World Cup appearances:")
        top_teams = agg_df.nlargest(10, 'Appearances')[[team_col, 'Appearances']]
        for _, row in top_teams.iterrows():
            print(f" {row[team_col]}: {int(row['Appearances'])} appearances")

    # Find goals columns
    goals_cols = [col for col in agg_df.columns if 'goal' in str(col).lower() and 'per_game' not in str(col).lower()]
    if goals_cols:
        goals_col = goals_cols[0]
        print(f"\nTop scoring teams (total {goals_col}):")
        top_scorers = agg_df.nlargest(10, goals_col)[[team_col, goals_col]]
        for _, row in top_scorers.iterrows():
            print(f" {row[team_col]}: {int(row[goals_col])} goals")

    # Find points columns
    points_cols = [col for col in agg_df.columns if 'pts' in str(col).lower() or 'point' in str(col).lower()]
    if points_cols:
        points_col = points_cols[0]
        print(f"\nTop teams by total {points_col}:")
        top_points = agg_df.nlargest(10, points_col)[[team_col, points_col]]
        for _, row in top_points.iterrows():
            print(f" {row[team_col]}: {int(row[points_col])} points")

    # Show data structure
    print(f"\nFinal dataset structure:")
    print(f"  - Total teams: {len(agg_df)}")
    print(f"  - Total columns: {len(agg_df.columns)}")
    print(f"  - Key columns: {list(agg_df.columns[:8])}...")

# Main execution
if __name__ == "__main__":
    input_file = "combined_world_cup_standings.xlsx"  

    try:
        print("Starting Team Statistics Aggregation...")
        print("=" * 60)

        aggregated_data = aggregate_team_stats(input_file)

        print("\n" + "=" * 60)
        print("AGGREGATION COMPLETE!")
        print("=" * 60)

        # Show analysis
        analyze_aggregated_data(aggregated_data)

        # Show sample of final data
        team_col = identify_team_column(aggregated_data)
        print(f"\n Sample of aggregated data (first 5 teams):")
        sample_cols = [team_col] + [col for col in ['Appearances', 'MP', 'W', 'D', 'L', 'GF', 'GA', 'Pts'] if col in aggregated_data.columns]
        print(aggregated_data[sample_cols].head())

    except FileNotFoundError:
        print(f"File '{input_file}' not found!")
        print("Please make sure the combined Excel file exists and update the 'input_file' variable.")
    except Exception as e:
        print(f"Error: {e}")

In [3]:
df = pd.read_excel("aggregated_team_stats.xlsx")

# Add features
df['Appearances'] = df['Year'].str.count('WC_')

df['win_rate'] = (df['W'] / df['MP']).round(3)

df['goal_ratio'] = (df['GF'] / (df['GA'] + 0.1)).round(2)

df['Appearances'] = df['Year'].str.count('WC_')

df['world_cup_experience'] = df['Appearances']

df['points_per_game'] = (df['Pts'] / df['MP']).round(2)

df['goal_diff_per_game'] = (df['GD'] / df['MP']).round(2)

df['attack_power'] = df['GF_per_game'] * df['goal_ratio']

df['defense_strength'] = 1 / (df['GA_per_game'] + 0.1)  # Avoid division by zero

df['team_consistency'] = df['W_per_game'] + (df['D_per_game'] * 0.5)

df['performance_efficiency'] = df['points_per_game'] / df['MP']

df.drop(columns=['Notes','Top Team Scorer','Goalkeeper',], inplace=True)

df.to_excel("aggregated_team_stats.xlsx", index=False)

print("All features added to the file!")

All features added to the file!
